In [171]:
import numpy as np 
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
import seaborn as sns
import random 
import warnings
import operator
warnings.filterwarnings("ignore")

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [2]:
schl_df = pd.read_csv("2016 School Explorer.csv")
reg_df = pd.read_csv("D5 SHSAT Registrations and Testers.csv")

In [122]:
class data_chart():
    """ 
    Class to visualize data features
    """
    def __init__(self, df, column_name, data_type):
        """
        Initialize class
            data_type: 'cat' or 'value'
        """
        self.column_name = column_name
        self.column_data = df[column_name]
        if data_type == 'cat':
            self.cat_data()
        elif data_type == 'value':
            self.value_data()
        else:
            raise NotImplementedError
        self.data_type = data_type
        
    def cat_data(self):
        nlst, clst = [], []
        cdict = self.column_data.value_counts().to_dict()
        cdict = sorted(cdict.items(), key=operator.itemgetter(1))
        for tup in cdict:
            nlst.append(tup[0])
            clst.append(tup[1])
        self.nlst = nlst
        self.clst = clst
        
    def value_data(self):
        self.values = self.column_data
        
    def plot(self, cat='bar', **kwargs):
        """
        Visualize data
        """
        if self.data_type == 'cat':
            fig = self.plot_cat(cat=cat, **kwargs)
        elif self.data_type == 'value':
            fig = self.plot_value(**kwargs)
            
        py.iplot(fig, filename=f'plot_of_{self.column_name}')
        
    def plot_cat(self, cat, **kwargs):
        data_bar = {"y": self.nlst, "x": self.clst, "width": 0.9, "opacity": 0.6, "orientation": 'h', "type": 'bar'}
        data_bar['hovertext'] = list(map(lambda i: f'{i/sum(self.clst)*100:.2f}%', self.clst))
        
        data_pie = {'values': self.clst, "labels": self.nlst, "hoverinfo": 'label+percent', 
                    "type": 'pie', "hole": .3}
        
        layout = {"title": self.column_name, "autosize": True,
                  "paper_bgcolor":'rgb(243, 243, 243)', "plot_bgcolor":'rgb(243, 243, 243)'}
    
        if 'data_bar' in kwargs:
            data_bar.update(kwargs['data_bar'])
            
        if 'data_pie' in kwargs:
            data_pie.update(kwargs['data_pie'])   
            
        if 'layout' in kwargs:
            layout.update(kwargs['layout'])      
        if cat == 'bar':
            return {"data": [data_bar], "layout": layout}
        elif cat == 'pie':
            return {"data": [data_pie], "layout": layout}
        else:
            raise NotImplementedError
            
    def plot_value(self, **kwargs):
        data_hist = {"y": self.values, "name": self.column_name, "type": 'histogram'}   
        data_box  = {"y": self.values, "name": 'Mean & SD', 'type': 'box', 'boxmean': 'sd'}
        layout = {"title": f'Data analysis for {self.column_name}', 
                  "paper_bgcolor":'rgb(243, 243, 243)', "plot_bgcolor":'rgb(243, 243, 243)',
                  "showlegend": False, 
                  'yaxis': {'title': self.column_name}, 
                  'xaxis1': {'title': 'count'}, 
                  'xaxis2': {'title': f'Mean = {np.mean(self.values):.4f}'}}
        
        if 'data_hist' in kwargs:
            data_hist.update(kwargs['data_hist'])
        
        if 'data_box' in kwargs:
            data_box.update(kwargs['data_box'])
            
        if 'layout' in kwargs:
            layout.update(kwargs['layout'])
            
        fig = tls.make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=('Histogram', 'Boxplot'))
        fig.append_trace(data_hist, 1, 1)
        fig.append_trace(data_box, 1, 2)
      
        fig['layout'].update(layout)
        
        return fig

<h2>School City</h2>
Lets have a look at the distribution of schools in the cities given in the database.

In [123]:
school_city = data_chart(schl_df, 'City', 'cat')
margin = go.Margin(l=250, r=50, b=100, t=100, pad=20)
school_city.plot(**{'layout':{'margin': margin, 'height':800}})

Well, According to the above charts, Most of the Schools are from Brooklyn(32.3%), followed by Bronx(23.3%), New York(18.2%), Staten Island(4.72%), Jamaica(2.52%)

In [124]:
school_city.plot('pie')

<h2>School Income Estimate</h2>
Let's have a look at how much estimated income do the schools have and how are they distributed. The column is contains string values so, need to change it to float values and plot some more charts.

In [125]:
def get_dollar_amount(x):
    x = str(x)
    if x == 'nan':
        return np.nan
    else:
        return float(x.strip().replace('$', '').replace(',',''))
    
schl_df['School Income Estimate'] = schl_df['School Income Estimate'].apply(get_dollar_amount)

In [126]:
school_sie = data_chart(schl_df, 'School Income Estimate', 'value')
school_sie.plot()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



Most of the colleges have income from 25k to 45k, There are a few colleges with extremely high incomes like 140k and 180k, Would like to check if they are Community colleges or not. 

<h2>Community College or Not</h2>
How many colleges from the given dataset are Community colleges?

In [127]:
sch_cmnty = data_chart(schl_df, 'Community School?', 'cat')
sch_cmnty.plot('pie', **{'data_pie': {'hole': 0}})

Out of the given 1272 colleges, only 76 of them are Community colleges.

<h2>High Grade</h2>
Let's see the highest grade offered by the Schools.

In [128]:
high_grd = data_chart(schl_df, 'Grade High', 'cat')
x = list(map(lambda x: f'Grade {x}', high_grd.nlst))
y = high_grd.clst
high_grd.plot(**{'data_bar': {'x': x, 'y': y, 'orientation': 'v', 'width': 0.5},
                 'layout': {'width': 600, 'xaxis': dict(tickangle=-45)}
                })

<h2>Economic Need Index</h2>
The Economic Need Index reflects the socioeconomics of the school population. It is calculated using the following formula:

Economic Need Index = (Percent Temporary Housing) + (Percent
HRA-eligible * 0.5) + (Percent Free Lunch Eligible * 0.5)

In [129]:
schl_df['Economic Need Index'].isnull().sum()

25

In [130]:
sch_eni = data_chart(schl_df, 'Economic Need Index', 'value')
sch_eni.plot()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



<h2>Framework for Great Schools</h2>
The Framework for Great Schools sets forth six elements—Rigorous Instruction,
Collaborative Teachers, Supportive Environment, Effective School Leadership,
Strong Family-Community Ties, and Trust—that drive student achievement and
school improvement.

The School Quality Reports share ratings and information on how schools are
performing on the six Framework elements.

1. **Rigorous Instruction**
2. **Collaborative Teachers**
3. **Supportive Environment**
4. **Effective School Leadership**
5. **Strong Family-Community Ties**
6. **Trust**

In [131]:
def rating_data(column_name):
    column_name += ' Rating'
    dct = schl_df[column_name].value_counts().to_dict()
    return go.Bar(
                y = list(dct.values()),
                x = list(dct.keys()),
                width = 0.9,
                opacity = 0.6, 
                orientation = 'v', 
                name = column_name,
                marker = dict(color='rgb(158,202,225)', line=dict(color='rgb(8,48,107)', width=1.5))
            )

def ratio_data(column_name):
    column_name += ' %'
    return go.Histogram(x = schl_df[column_name], name = column_name)

In [132]:
cols = ['Rigorous Instruction', 'Collaborative Teachers', 'Supportive Environment',
        'Effective School Leadership', 'Strong Family-Community Ties', 'Trust']

In [133]:
st = []
for c in cols:
    st.append(f'{c} Rating')
    st.append(f'{c} %')
st = tuple(st)

fig = tls.make_subplots(rows=6, cols=2, subplot_titles=st)

i = 1
for c in cols:
    fig.append_trace(rating_data(c), i, 1)
    fig.append_trace(ratio_data(c), i, 2)
    i += 1


fig['layout'].update(height=2400,title='School Quality Report Charts', showlegend=False, 
                     paper_bgcolor='rgb(243, 243, 243)',plot_bgcolor='rgb(243, 243, 243)')
py.iplot(fig, filename='simple-subplot')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]  
[ (2,1) x3,y3 ]    [ (2,2) x4,y4 ]  
[ (3,1) x5,y5 ]    [ (3,2) x6,y6 ]  
[ (4,1) x7,y7 ]    [ (4,2) x8,y8 ]  
[ (5,1) x9,y9 ]    [ (5,2) x10,y10 ]
[ (6,1) x11,y11 ]  [ (6,2) x12,y12 ]



<h2>Student Achievement Rating</h2>

In [134]:
sch_sar = data_chart(schl_df, 'Student Achievement Rating', 'cat')
sch_sar.plot('pie')

<h2>Average Math and ELA Proficiency</h2>
Understanding Proficiency provides resources that guide educators in analyzing student work on performance tasks in order to develop a deeper understanding of the Maths/English Language Arts (ELA)/Literacy Common Core State Standards

In [135]:
ave_math = data_chart(schl_df, 'Average Math Proficiency', 'value')
ave_math.plot()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [136]:
ave_ela = data_chart(schl_df, 'Average ELA Proficiency', 'value')
ave_ela.plot()

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



<h2>Students Chronically Absent</h2>

In [148]:
def convert_percent(x):
    x = str(x)
    if x == 'nan':
        return np.nan
    else:
        return float(x.strip().replace('%', ''))

schl_df['Percent of Students Chronically Absent']  = schl_df['Percent of Students Chronically Absent'].apply(convert_percent)

In [161]:
ave_ela = data_chart(schl_df, 'Percent of Students Chronically Absent', 'value')
ave_ela.plot(**{'layout': {'title': f'Data analysis {ave_ela.column_name} (%)', 
                           'yaxis':{'title': f'{ave_ela.column_name} (%)'},
                           'xaxis2': {'title': f'Mean = {np.mean(ave_ela.values):.2f} (%)'}
                          }
               })

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



In [162]:
def p2f(x):
    return float(x.strip('%'))/100

schl_df['Percent Asian'] = schl_df['Percent Asian'].apply(p2f)
schl_df['Percent Black'] = schl_df['Percent Black'].apply(p2f)
schl_df['Percent Hispanic'] = schl_df['Percent Hispanic'].apply(p2f)
schl_df['Percent White'] = schl_df['Percent White'].apply(p2f)
schl_df['Percent Black / Hispanic'] = schl_df['Percent Black / Hispanic'].apply(p2f)
schl_df['Percent ELL'] = schl_df['Percent ELL'].apply(p2f)

In [163]:
d3 = pd.DataFrame(schl_df.groupby(['City']).mean())
#d3[['Economic Need Index','School Income Estimate','Percent Asian','Percent Black','Percent Hispanic','Percent Black / Hispanic','Percent White']]
#d3.head(25)

<h2>Scores for ELA and Maths from Grade 3 to Grade 8</h2>
**Numeric Grade Scores** : Report Cards Give Up A’s and B’s for 4s and 3s. The lowest mark, 1, indicates a student is not meeting New York State’s academic standards, while the top grade of 4 celebrates “meeting standards with distinction.”

Below plots are bubble charts for the Number of students getting 4 Scores in Grades 3 to 8.  

In [213]:
schl_df.rename(index=str, 
               columns={"Grade 3 Math - All Students tested": "Grade 3 Math - All Students Tested"},
               inplace=True)

In [304]:
# Create a trace
colors = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, 12)]
race = ['All Students','American Indian or Alaska Native','Black or African American',
          'Hispanic or Latino','Asian or Pacific Islander','White',
          'Multiracial','Limited English Proficient','Economically Disadvantaged']
subject = ['ELA', 'Math']

grade = 6
all_trace = {}
for s in subject:
    schl_cnt = []
    stdnt_cnt = []
    stdnt_pccnt = []
    tot_stdnt = schl_df[f'Grade {grade} {s} - All Students Tested'].sum()
    print(tot_stdnt)
    for r in race:
        schl_cnt.append(len(schl_df[schl_df[f'Grade 3 {s} 4s - {r}'] > 0]))
        stdnt_cnt.append(schl_df[f'Grade 3 {s} 4s - {r}'].sum())
        stdnt_pccnt.append(schl_df[f'Grade 3 {s} 4s - {r}'].sum()/tot_stdnt)

    trace = go.Scatter(
                    x=race,
                    y=schl_cnt,
                    mode='markers',
                    name = f'Grade{grade}-{s}',
                    hovertext=list(map(lambda i: f'{stdnt_cnt[i]} students in {schl_cnt[i]} schools<br>{stdnt_pccnt[i]*100:.2f}% out of all student tested', 
                                    range(len(race)))),
                    marker=dict(size=[x*150 + 20 for x in stdnt_pccnt],
                               color=colors[:len(race)],)
                        )
    all_trace[s] = trace
        
fig = tls.make_subplots(rows=1, cols=2, shared_yaxes=True, subplot_titles=(f'ELA', f'Math'));
fig.append_trace(all_trace['ELA'], 1, 1);
fig.append_trace(all_trace['Math'], 1, 2);

fig['layout'].update(height=500, title=f'Count of Students scoring 4s in Grade {grade}', 
                     showlegend=False,paper_bgcolor='rgb(243, 243, 243)',
                     yaxis1 = dict(title='Number of schools'),
                     yaxis2 = dict(title='Number of schools'), 
                     plot_bgcolor='rgb(243, 243, 243)',
                     margin = go.Margin(l=100, r=120, b=150, t=100, pad=20));

py.iplot(fig, filename='Proficiency-subplot')

69383
70540
This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y1 ]



<h2>2016 School Explorer</h2>
Lets have a look at the second csv file given to us and explore it a bit. 

<h2>Number of students who registered for the SHSAT</h2>


In [307]:
d4 = reg_df[reg_df['Year of SHST'] == 2013]
d4 = pd.DataFrame(d4.groupby(['School name']).sum()).reset_index()
d5 = reg_df[reg_df['Year of SHST'] == 2014]
d5 = pd.DataFrame(d5.groupby(['School name']).sum()).reset_index()
d6 = reg_df[reg_df['Year of SHST'] == 2015]
d6 = pd.DataFrame(d6.groupby(['School name']).sum()).reset_index()
d7 = reg_df[reg_df['Year of SHST'] == 2016]
d7 = pd.DataFrame(d7.groupby(['School name']).sum()).reset_index()

d4['Number of students who did not take the SHSAT after registering'] = d4['Number of students who registered for the SHSAT'] - d4['Number of students who took the SHSAT']
d5['Number of students who did not take the SHSAT after registering'] = d5['Number of students who registered for the SHSAT'] - d5['Number of students who took the SHSAT']
d6['Number of students who did not take the SHSAT after registering'] = d6['Number of students who registered for the SHSAT'] - d6['Number of students who took the SHSAT']
d7['Number of students who did not take the SHSAT after registering'] = d7['Number of students who registered for the SHSAT'] - d7['Number of students who took the SHSAT']

In [308]:

trace1 = go.Bar(
    y=reg_df['School name'],
    x=reg_df['Number of students who registered for the SHSAT'],
    name='Number of students who registered for the SHSAT',
    orientation = 'h'
)
trace2 = go.Bar(
    y=reg_df['School name'],
    x=reg_df['Number of students who took the SHSAT'],
    name='Number of students who took the SHSAT',
    orientation = 'h'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack',
    showlegend = False,
    margin=go.Margin(
        l=350,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    height = 800,
    
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')


<h2>Number of students who registered for the SHSAT</h2>
Lets have a look at the percentage of students who registered and took the SHSAT.

In [309]:
label_common = ['Number of students who did not take the SHSAT after registering', 'Number of students who took the SHSAT']

values_13 = [np.sum(d4['Number of students who did not take the SHSAT after registering']), np.sum(d4['Number of students who took the SHSAT'])]
values_14 = [np.sum(d5['Number of students who did not take the SHSAT after registering']), np.sum(d5['Number of students who took the SHSAT'])]
values_15 = [np.sum(d6['Number of students who did not take the SHSAT after registering']), np.sum(d6['Number of students who took the SHSAT'])]
values_16 = [np.sum(d7['Number of students who did not take the SHSAT after registering']), np.sum(d7['Number of students who took the SHSAT'])]


labels1 = ['Number of students who registered for the SHSAT','Number of students who took the SHSAT','Number of students who did not take the SHSAT']
val_2013 = [[np.sum(d4['Number of students who registered for the SHSAT']), 
             np.sum(d4['Number of students who took the SHSAT']),
             np.sum(d4['Number of students who did not take the SHSAT after registering'])]]

val_2014 = [[np.sum(d5['Number of students who registered for the SHSAT']), 
             np.sum(d5['Number of students who took the SHSAT']),
             np.sum(d5['Number of students who did not take the SHSAT after registering'])]]

val_2015 = [[np.sum(d6['Number of students who registered for the SHSAT']), 
             np.sum(d6['Number of students who took the SHSAT']),
             np.sum(d6['Number of students who did not take the SHSAT after registering'])]]

val_2016 = [[np.sum(d7['Number of students who registered for the SHSAT']), 
             np.sum(d7['Number of students who took the SHSAT']),
             np.sum(d7['Number of students who did not take the SHSAT after registering'])]]
trace0 = go.Bar(
    y=labels1,
    x=val_2013[0],
    marker=dict(color=['blue', 'yellow','red']),
    orientation ='h'
)

trace1 = go.Bar(
    x=val_2014[0],
    y=labels1,
    marker=dict(color=['blue', 'yellow','red']),
    orientation ='h'
)
fig = tls.make_subplots(rows=2, cols=1, subplot_titles=('Number of students who registered for the SHSAT for 2013', 'Number of students who registered for the SHSAT for 2014'));
fig.append_trace(trace0, 1, 1);
fig.append_trace(trace1, 2, 1);

fig['layout'].update(title = 'Number of students who registered for the SHSAT for 2013 and 2014', height=600, showlegend=False, margin=go.Margin(
        l=350,
        r=50,
        b=100,
        t=100,
        pad=4
    ),paper_bgcolor='rgb(243, 243, 243)',plot_bgcolor='rgb(243, 243, 243)');
py.iplot(fig, filename='Proficiency-subplot')

fig = {
  "data": [
    {
      "values": values_13,
      "labels": label_common,
      "domain": {"x": [0, .48]},
      "name": "2013",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": values_14,
      "labels": label_common,
      "text":["2013"],
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "2014",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Number of students who registered for the SHSAT for 2013 and 2014",
        "showlegend": False,
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2013",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2014",
                "x": 0.8,
                "y": 0.5
            }
        ],
        "paper_bgcolor": 'rgb(243, 243, 243)',"plot_bgcolor":'rgb(243, 243, 243)',
    }
}
py.iplot(fig, filename='donut')


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [310]:
trace0 = go.Bar(
    y=labels1,
    x=val_2015[0],
    marker=dict(color=['blue', 'yellow','red']),
    orientation ='h'
)

trace1 = go.Bar(
    x=val_2016[0],
    y=labels1,
    marker=dict(color=['blue', 'yellow','red']),
    orientation ='h'
)
fig = tls.make_subplots(rows=2, cols=1, subplot_titles=('Number of students who registered for the SHSAT for 2015', 'Number of students who registered for the SHSAT for 2016'));
fig.append_trace(trace0, 1, 1);
fig.append_trace(trace1, 2, 1);

fig['layout'].update(title = 'Number of students who registered for the SHSAT', height=600, showlegend=False, margin=go.Margin(
        l=350,
        r=50,
        b=100,
        t=100,
        pad=4
    ),paper_bgcolor='rgb(243, 243, 243)',plot_bgcolor='rgb(243, 243, 243)');
py.iplot(fig, filename='Proficiency-subplot')

fig = {
  "data": [
    {
      "values": values_15,
      "labels": label_common,
      "domain": {"x": [0, .48]},
      "name": "2015",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    },
    {
      "values": values_16,
      "labels": label_common,
      "text":["2016"],
      "textposition":"inside",
      "domain": {"x": [.52, 1]},
      "name": "2016",
      "hoverinfo":"label+percent+name",
      "hole": .4,
      "type": "pie"
    }],
  "layout": {
        "title":"Number of students who registered for the SHSAT for 2015 and 2016",
        "showlegend": False,
        "annotations": [
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2015",
                "x": 0.20,
                "y": 0.5
            },
            {
                "font": {
                    "size": 20
                },
                "showarrow": False,
                "text": "2016",
                "x": 0.8,
                "y": 0.5
            }
        ],
      "paper_bgcolor": 'rgb(243, 243, 243)',"plot_bgcolor":'rgb(243, 243, 243)',
    }
}
py.iplot(fig, filename='donut')



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [311]:
trace1 = go.Bar(
    x=[np.sum(d4['Enrollment on 10/31']), np.sum(d5['Enrollment on 10/31']), np.sum(d6['Enrollment on 10/31']), np.sum(d7['Enrollment on 10/31'])],
    y=['2013', '2014', '2015', '2016'],
    marker=dict(color=['blue', 'yellow','red', 'orange']),
    orientation ='h'
)
data = [trace1]

layout = go.Layout(
    title = "Enrollment on 10/31 Bar Plot",
    barmode='stack',
    height = 400,
     xaxis=dict(
        title='Count',
    ),
    yaxis=dict(
        title='Year',
    ),paper_bgcolor='rgb(243, 243, 243)',plot_bgcolor='rgb(243, 243, 243)')


fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='marker-h-bar')